- Reference https://minimin2.tistory.com/100

1. tfds의 데이터 셋 사용(MNIST)
2. Seqence 사용하여 custom dataset 만들기
3. Custom dataset 활용하여 CNN 학습

## 1. tfds의 데이터 셋 사용(MNIST)

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

In [3]:
# list_builders
tfds.list_builders() # 등록된 데이터셋 리스트 조회

['abstract_reasoning',
 'accentdb',
 'aeslc',
 'aflw2k3d',
 'ag_news_subset',
 'ai2_arc',
 'ai2_arc_with_ir',
 'amazon_us_reviews',
 'anli',
 'arc',
 'bair_robot_pushing_small',
 'bccd',
 'beans',
 'big_patent',
 'bigearthnet',
 'billsum',
 'binarized_mnist',
 'binary_alpha_digits',
 'blimp',
 'bool_q',
 'c4',
 'caltech101',
 'caltech_birds2010',
 'caltech_birds2011',
 'cars196',
 'cassava',
 'cats_vs_dogs',
 'celeb_a',
 'celeb_a_hq',
 'cfq',
 'chexpert',
 'cifar10',
 'cifar100',
 'cifar10_1',
 'cifar10_corrupted',
 'citrus_leaves',
 'cityscapes',
 'civil_comments',
 'clevr',
 'clic',
 'clinc_oos',
 'cmaterdb',
 'cnn_dailymail',
 'coco',
 'coco_captions',
 'coil100',
 'colorectal_histology',
 'colorectal_histology_large',
 'common_voice',
 'coqa',
 'cos_e',
 'cosmos_qa',
 'covid19sum',
 'crema_d',
 'curated_breast_imaging_ddsm',
 'cycle_gan',
 'deep_weeds',
 'definite_pronoun_resolution',
 'dementiabank',
 'diabetic_retinopathy_detection',
 'div2k',
 'dmlab',
 'downsampled_imagenet',
 

In [4]:
# split을 통한 train/test 별개로 데이터셋 로드 - 바로 split하여 load가능
train_dataset, train_info = tfds.load(name='mnist', split=tfds.Split.TRAIN, with_info=True)
test_dataset, test_info = tfds.load(name='mnist', split=tfds.Split.TEST, with_info=True)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.



Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]


Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [5]:
train_info

tfds.core.DatasetInfo(
    name='mnist',
    version=3.0.1,
    description='The MNIST database of handwritten digits.',
    homepage='http://yann.lecun.com/exdb/mnist/',
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=70000,
    splits={
        'test': 10000,
        'train': 60000,
    },
    supervised_keys=('image', 'label'),
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
    redistribution_info=,
)

In [6]:
for data in train_dataset:
  # ['image', 'label'], 둘 다 Tensor type
  print("data['image'] shape: ", data['image'].shape)
  print("max of data['image']: ", np.max(data['image']))
  print("min of data['image']: ", np.min(data['image']))
  print("data['image'] type: ", type(data['image']))
  print(data['label'])
  break

data['image'] shape:  (28, 28, 1)
max of data['image']:  255
min of data['image']:  0
data['image'] type:  <class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor(4, shape=(), dtype=int64)


In [7]:
train_data = list(train_dataset)
test_data = list(test_dataset)

In [8]:
train_data[0].keys()

dict_keys(['image', 'label'])

In [9]:
test_data[0].keys()

dict_keys(['image', 'label'])

In [10]:
len(train_data)

60000

In [11]:
len(test_data)

10000

## 2. Sequence 사용하여 custom datset 만들기

In [12]:
from tensorflow.keras.utils import Sequence
import math

Sequence를 활용하려면 __init__, __len__, __getitem__ 함수 세 가지는 필수적으로 만들어 주어야 함

In [13]:
class Dataloader(Sequence):
  def __init__(self, dataset, batch_size, shuffle=False):
    self.dataset = dataset
    self.indices = np.arange(len(self.dataset))
    self.batch_size = batch_size
    self.shuffle=shuffle
    self.on_epoch_end()
  
  def __len__(self):
    """데이터 로더의 전체 길이를 반환"""
    return math.ceil(len(self.dataset) / self.batch_size)

  def __getitem__(self, idx):
    # batch 단위로 직접 묶어줘야 함
    # sampler의 역할(index를 batch_size만큼 sampling해줌)
    indices = self.indices[idx*self.batch_size:(idx+1)*self.batch_size]

    # numpy 형태로 mini batch get, 수동으로 norm
    batch_x = [self.dataset[i]['image'].numpy()/255. for i in indices] 
    batch_y = [self.dataset[i]['label'].numpy() for i in indices] 

    return np.array(batch_x), np.array(batch_y)

  # epoch이 끝날때마다 실행
  def on_epoch_end(self):
    if self.shuffle == True:
        np.random.shuffle(self.indices)
        print('on_epoch_end operation')

In [14]:
BATCH_SIZE = 128
train_loader = Dataloader(train_data, BATCH_SIZE, shuffle=True)
valid_loader = Dataloader(test_data, BATCH_SIZE)

on_epoch_end operation


In [15]:
x, y = train_loader.__getitem__(0)
x.shape, y.shape

((128, 28, 28, 1), (128,))

## 3. Custom dataset 활용하여 CNN 학습

In [16]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Activation, Conv2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

class MyModel(Model):
  def __init__(self, input_height, input_width, input_channel):
    super(MyModel, self).__init__()

    inputs = Input((input_height, input_width, input_channel))
    x = Conv2D(32, (3, 3), padding='same')(inputs)
    x = Activation('relu')(x)

    x = Conv2D(64, (3, 3), padding='same')(x)
    x = Activation('relu')(x)

    x = Flatten()(x)
    x = Dense(10)(x)

    outputs = Activation('softmax')(x)

    self.model = Model(inputs = inputs, outputs = outputs)
    self.model.summary()

  def call(self, x):
    return self.model(x)

In [17]:
model = MyModel(28,28,1)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 activation (Activation)     (None, 28, 28, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 64)        18496     
                                                                 
 activation_1 (Activation)   (None, 28, 28, 64)        0         
                                                                 
 flatten (Flatten)           (None, 50176)             0         
                                                                 
 dense (Dense)               (None, 10)                501770

### tf.GradientTape()를 사용하여 Training
* dataloader에서 on_epoch_end은 자동으로 동작 X(수동으로 epoch 끝날 때마다 실행 시켜줘야 함)

In [18]:
opt = tf.keras.optimizers.Adam(1e-3)

# integer형태 계산, one-hot vector를 사용하려면 -> CategoricalCrossentropy
# softmax 함수를 거치므로 frome_logits = False(default)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False) 
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy') # integer형태 계산, one-hot vector를 사용하려면 -> CategoricalAccuracy


def train_step(net, x_data, y_data, optimizer):
  """Trains `net` on `example` using `optimizer`."""
  with tf.GradientTape() as tape:
    predictions = net(x_data, training=True)
    loss = loss_object(y_data, predictions)
  grads = tape.gradient(loss, net.trainable_variables)
  optimizer.apply_gradients(zip(grads, net.trainable_variables))

  train_accuracy(y_data, predictions)
  train_loss(loss)

  return loss

In [19]:
EPOCHS = 3

In [20]:
def train(net, opt, train_loader):  
  for epoch in range(EPOCHS):
    epoch_loss = []
    for x, y in train_loader:
      x_data, y_data = x, y
      loss = train_step(net, x_data, y_data, opt)
      epoch_loss.append(loss)

    template = 'Epoch {}, Loss: {}, Accuracy: {}'
    print('')
    print (template.format(epoch+1,np.mean(epoch_loss), train_accuracy.result()*100))
    
    # Reset metrics every epoch - 매 epoch마다 reset
    train_loss.reset_states()
    train_accuracy.reset_states()

In [21]:
train(model, opt, train_loader)


Epoch 1, Loss: 0.1756250411272049, Accuracy: 94.85499572753906

Epoch 2, Loss: 0.05198778584599495, Accuracy: 98.45333099365234

Epoch 3, Loss: 0.035208847373723984, Accuracy: 98.97000122070312


### compile and fit()을 이용한 training
* on_epoch_end이 epoch 마다 자동 실행

In [22]:
model_fit = MyModel(28,28,1)
model_fit.compile(optimizer="adam", loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False) , metrics = ['acc'])

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 32)        320       
                                                                 
 activation_3 (Activation)   (None, 28, 28, 32)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 28, 28, 64)        18496     
                                                                 
 activation_4 (Activation)   (None, 28, 28, 64)        0         
                                                                 
 flatten_1 (Flatten)         (None, 50176)             0         
                                                                 
 dense_1 (Dense)             (None, 10)                5017

In [23]:
model_fit.fit(train_loader, epochs=EPOCHS)

Epoch 1/3
469/469 [==============================] - 8s 15ms/step - loss: 0.1664 - acc: 0.9520
on_epoch_end operation
Epoch 2/3
469/469 [==============================] - 7s 15ms/step - loss: 0.0536 - acc: 0.9838
on_epoch_end operation
Epoch 3/3
469/469 [==============================] - 7s 15ms/step - loss: 0.0364 - acc: 0.9887
on_epoch_end operation
